## EEG Classification CNN ML Model (Control vs Schizophrenia)

Graph-based analysis of brain connectivity in schizophrenia - Elzbieta Olejarczyk , Wojciech Jernajczyk
Research Paper:  https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0188629
Dataset:         https://repod.icm.edu.pl/dataset.xhtml?persistentId=doi:10.18150/repod.0107441

In [98]:
# Necessary Imports

from glob import glob # Read files from folder
import os # Operating System interfaces

import mne # Analyze EEG data

import numpy as np # Calculations
import pandas as pd # Dataframes

import matplotlib.pyplot as plt # Plotting and Visualizations

from scipy import stats # statistical calculations - scientific data

from sklearn.linear_model import LogisticRegression # Logistic Regression for Features array
from sklearn.pipeline import Pipeline # Use a pipeline of our scaling for classification purpose
from sklearn.preprocessing import StandardScaler # Use standar scaler for classification preprocessing
from sklearn.model_selection import GroupKFold # Use KFold for groups(control vs schizophrenic)
from sklearn.model_selection import GridSearchCV # Hyperparameter Tuning - Speed up performance
from sklearn.model_selection import LeaveOneGroupOut # Ability to exclude a group

# TensorFlow Keras Imports
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPool1D,\
    GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [99]:
# Access all edf files from directory
all_file_path = glob('EEG_Schizophrenia_Data/*.edf')
all_file_path

['EEG_Schizophrenia_Data\\h01.edf',
 'EEG_Schizophrenia_Data\\h02.edf',
 'EEG_Schizophrenia_Data\\h03.edf',
 'EEG_Schizophrenia_Data\\h04.edf',
 'EEG_Schizophrenia_Data\\h05.edf',
 'EEG_Schizophrenia_Data\\h06.edf',
 'EEG_Schizophrenia_Data\\h07.edf',
 'EEG_Schizophrenia_Data\\h08.edf',
 'EEG_Schizophrenia_Data\\h09.edf',
 'EEG_Schizophrenia_Data\\h10.edf',
 'EEG_Schizophrenia_Data\\h11.edf',
 'EEG_Schizophrenia_Data\\h12.edf',
 'EEG_Schizophrenia_Data\\h13.edf',
 'EEG_Schizophrenia_Data\\h14.edf',
 'EEG_Schizophrenia_Data\\s01.edf',
 'EEG_Schizophrenia_Data\\s02.edf',
 'EEG_Schizophrenia_Data\\s03.edf',
 'EEG_Schizophrenia_Data\\s04.edf',
 'EEG_Schizophrenia_Data\\s05.edf',
 'EEG_Schizophrenia_Data\\s06.edf',
 'EEG_Schizophrenia_Data\\s07.edf',
 'EEG_Schizophrenia_Data\\s08.edf',
 'EEG_Schizophrenia_Data\\s09.edf',
 'EEG_Schizophrenia_Data\\s10.edf',
 'EEG_Schizophrenia_Data\\s11.edf',
 'EEG_Schizophrenia_Data\\s12.edf',
 'EEG_Schizophrenia_Data\\s13.edf',
 'EEG_Schizophrenia_Data\\s1

In [100]:
# Use list comprehension to split into healthy and patient file paths
# In Data: h ~ healthy ; s ~ schizophrenia
healthy_file_path = [i for i in all_file_path if 'h' in i.split('\\')[1]]
patient_file_path = [i for i in all_file_path if 's' in i.split('\\')[1]]

In [101]:
healthy_file_path

['EEG_Schizophrenia_Data\\h01.edf',
 'EEG_Schizophrenia_Data\\h02.edf',
 'EEG_Schizophrenia_Data\\h03.edf',
 'EEG_Schizophrenia_Data\\h04.edf',
 'EEG_Schizophrenia_Data\\h05.edf',
 'EEG_Schizophrenia_Data\\h06.edf',
 'EEG_Schizophrenia_Data\\h07.edf',
 'EEG_Schizophrenia_Data\\h08.edf',
 'EEG_Schizophrenia_Data\\h09.edf',
 'EEG_Schizophrenia_Data\\h10.edf',
 'EEG_Schizophrenia_Data\\h11.edf',
 'EEG_Schizophrenia_Data\\h12.edf',
 'EEG_Schizophrenia_Data\\h13.edf',
 'EEG_Schizophrenia_Data\\h14.edf']

In [102]:
patient_file_path

['EEG_Schizophrenia_Data\\s01.edf',
 'EEG_Schizophrenia_Data\\s02.edf',
 'EEG_Schizophrenia_Data\\s03.edf',
 'EEG_Schizophrenia_Data\\s04.edf',
 'EEG_Schizophrenia_Data\\s05.edf',
 'EEG_Schizophrenia_Data\\s06.edf',
 'EEG_Schizophrenia_Data\\s07.edf',
 'EEG_Schizophrenia_Data\\s08.edf',
 'EEG_Schizophrenia_Data\\s09.edf',
 'EEG_Schizophrenia_Data\\s10.edf',
 'EEG_Schizophrenia_Data\\s11.edf',
 'EEG_Schizophrenia_Data\\s12.edf',
 'EEG_Schizophrenia_Data\\s13.edf',
 'EEG_Schizophrenia_Data\\s14.edf']

In [103]:
# Print file path lengths
print("all_file_path length:",len(all_file_path))
print('healthy_file_path length: ',len(healthy_file_path))
print('patient_file_path length: ',len(patient_file_path))

all_file_path length: 28
healthy_file_path length:  14
patient_file_path length:  14


In [104]:
# Function to read data from file paths
def read_data(file_path):
    # Read in raw edf data from path
    data = mne.io.read_raw_edf(file_path, preload=True)
    
    # Define eeg references - use default - average of all channels as reference for other channels
    data.set_eeg_reference()
    
    # Filter data based on low and high frequency (in Hz)
    data.filter(l_freq=0.5, h_freq=45)
    
    # Convert data into epochs(segments), pass in data, signal duartion (5 secs), add overlap of segments 
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap=1)
    
    # Create array from epochs data
    array = epochs.get_data()
    
    return array

In [105]:
# Sample example data for 1 healhy participant
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from C:\Users\Peter\EEG_Schizophrenia_Data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 sec)

Not setting metadata
231 matching events found
No baseline correction applied
0 projection items activa

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.0s finished


In [106]:
sample_data.shape
# shape = (# of epochs, # of channels, length of signal)

(231, 19, 1250)

In [107]:
%%capture 
# Add this %%capture at top of cell to avoid printing all the data
# Read data and define epochs array for healthy control and schizoprenic patient
control_epochs_array = [read_data(i) for i in healthy_file_path]
patient_epochs_array = [read_data(i) for i in patient_file_path]

In [108]:
# Print epoch array shape
print("control_epochs_array shape:", control_epochs_array[0].shape, control_epochs_array[1].shape)
print("patient_epochs_array shape:", patient_epochs_array[0].shape, patient_epochs_array[1].shape)

control_epochs_array shape: (231, 19, 1250) (227, 19, 1250)
patient_epochs_array shape: (211, 19, 1250) (286, 19, 1250)


In [109]:
# Create labels for control and patient epoch arrays
control_epochs_labels = [len(i)*[0] for i in control_epochs_array]
patient_epochs_labels = [len(i)*[1] for i in patient_epochs_array]
len(control_epochs_labels), len(patient_epochs_array)

(14, 14)

In [110]:
# combine both epochs array lists into a signal array list
epochs_array = control_epochs_array + patient_epochs_array

# combine labels
epochs_labels = control_epochs_labels + patient_epochs_labels

In [111]:
# Split data on basis of subjects(participants), Not on the basis of trials or epochs
group_list = [[i]*len(j) for i, j in enumerate(epochs_array)]
print(f"group_list length: {len(group_list)}")

group_list length: 28


In [112]:
# Convert lists into arrays
data_array = np.vstack(epochs_array)
label_array = np.hstack(epochs_labels)
group_array = np.hstack(group_list)

print(data_array.shape, label_array.shape, group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


#### Feature Extraction Functions

In [113]:
def mean(x):
    return np.mean(x,axis=-1)

def std(x):
    return np.std(x,axis=-1)

def ptp(x):
    return np.ptp(x,axis=-1)

def var(x):
    return np.var(x,axis=-1)

def minim(x):
    return np.min(x,axis=-1)

def maxim(x):
    return np.max(x,axis=-1)

def argminim(x):
    return np.argmin(x,axis=-1)

def argmaxim(x):
    return np.argmax(x,axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2,axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)

def skewness(x):
    return stats.skew(x,axis=-1)

def kurtosis(x):
    return stats.kurtosis(x,axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argminim(x),
                           argmaxim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis=-1)

In [114]:
# Use functions for feature extraction
features = []
for d in data_array:
    features.append(concatenate_features(d))

In [115]:
# Convert features from list to array
features_array = np.array(features)
features_array.shape

(7201, 228)

In [116]:
# Define sklearn models
clf = LogisticRegression()
gkf = GroupKFold(5)

# Define pipeline - standard scaler - logistic regression
pipe = Pipeline([('scaler',StandardScaler()), ('clf',clf)])

# Define parameters for tuning
param_grid = {'clf__C':[0.1,0.5,0.7,1,3,5,7]}

# Define Grid Search object 
gscv = GridSearchCV(pipe, param_grid, cv=gkf, n_jobs=12)
gscv.fit(features_array, label_array, groups=group_array)

C:\Users\Peter\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [117]:
# Use gscv to find best score we achieved
gscv.best_score_

0.663034159305596

#### Deep Learning CNN

In [118]:
# Modify shape of data for CNN

# First convert data to numpy array
epochs_array = np.vstack(epochs_array)
epochs_labels = np.hstack(epochs_labels)
groups_array = np.hstack(group_list)

In [119]:
# Check shape for each array
print(epochs_array.shape)
print(epochs_labels.shape)
print(groups_array.shape)

# 7201 - # of segments, 19 - # of channels, 1250 - length

(7201, 19, 1250)
(7201,)
(7201,)


In [120]:
# In epoch array - move axis of channel to the end
epochs_array = np.moveaxis(epochs_array,1,2)
epochs_array.shape

(7201, 1250, 19)

In [121]:
# Deep 1D Convolutional Neural Network
def cnn_model():
    clear_session()
    model = Sequential()
    # Layer 1
    model.add(Conv1D(filters=5, kernel_size=3, strides=1, input_shape=(1250,19)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    # Layer 2
    model.add(MaxPool1D(pool_size=2, strides=2))
    # Layer 3
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    # Layer 4
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Dropout(0.5))
    # Layer 5
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    # Layer 6
    model.add(AveragePooling1D(pool_size=2,strides=2))
    model.add(Dropout(0.5))
    # Layer 7
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    # Layer 8
    model.add(AveragePooling1D(pool_size=2,strides=2))
    # Layer 9
    model.add(Conv1D(filters=5, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    # Layer 10
    model.add(GlobalAveragePooling1D())
    # Layer 11
    model.add(Dense(1,activation='sigmoid'))
    # Compile
    model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1248, 5)           290       
                                                                 
 batch_normalization (BatchN  (None, 1248, 5)          20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 624, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 622, 5)            80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 622, 5)            0

In [122]:
# Define group k fold
gkf = GroupKFold()

In [123]:
# List to store accuracy from each fold
accuracy=[]

In [124]:
# Iterate through train_index and val_index
for train_index, val_index in gkf.split(epochs_array, epochs_labels, groups=groups_array):
    # Define training and validation features + lables
    train_features, train_labels = epochs_array[train_index], epochs_labels[train_index]
    val_features, val_labels = epochs_array[val_index],epochs_labels[val_index]
    
    # Convert data from 3D to 2D for Scaling and Normalization
    # (5744, 1250, 19) -> (5744 * 1250, 19)
    # And then re-shape back to 3D
    scaler = StandardScaler()
    train_features = scaler.fit_transform\
        (train_features.reshape(-1,train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.fit_transform\
        (val_features.reshape(-1,val_features.shape[-1])).reshape(val_features.shape)
    
    # Call and fit model
    model = cnn_model()
    model.fit(train_features, train_labels, epochs=1000, batch_size=1024,
              validation_data=(val_features, val_labels))
    
    # Append model accuracy to list
    accuracy.append(model.evaluate(val_features, val_labels)[1])
    
    break

Epoch 1/1000
6/6 [==============================] - 2s 170ms/step - loss: 0.7260 - accuracy: 0.4067 - val_loss: 0.7042 - val_accuracy: 0.6266
Epoch 2/1000
6/6 [==============================] - 1s 139ms/step - loss: 0.7099 - accuracy: 0.4074 - val_loss: 0.7019 - val_accuracy: 0.2704
Epoch 3/1000
6/6 [==============================] - 1s 142ms/step - loss: 0.7001 - accuracy: 0.4152 - val_loss: 0.7018 - val_accuracy: 0.2780
Epoch 4/1000
6/6 [==============================] - 1s 147ms/step - loss: 0.6953 - accuracy: 0.4932 - val_loss: 0.7033 - val_accuracy: 0.3590
Epoch 5/1000
6/6 [==============================] - 1s 146ms/step - loss: 0.6904 - accuracy: 0.5749 - val_loss: 0.7055 - val_accuracy: 0.3706
Epoch 6/1000
6/6 [==============================] - 1s 142ms/step - loss: 0.6877 - accuracy: 0.5897 - val_loss: 0.7078 - val_accuracy: 0.3720
Epoch 7/1000
6/6 [==============================] - 1s 139ms/step - loss: 0.6857 - accuracy: 0.5902 - val_loss: 0.7102 - val_accuracy: 0.3713
Epoch 

6/6 [==============================] - 1s 137ms/step - loss: 0.2117 - accuracy: 0.9356 - val_loss: 3.4099 - val_accuracy: 0.3727
Epoch 116/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.2016 - accuracy: 0.9436 - val_loss: 3.4463 - val_accuracy: 0.3720
Epoch 117/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.2025 - accuracy: 0.9445 - val_loss: 3.6671 - val_accuracy: 0.3720
Epoch 118/1000
6/6 [==============================] - 1s 138ms/step - loss: 0.2051 - accuracy: 0.9398 - val_loss: 3.5014 - val_accuracy: 0.3720
Epoch 119/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.2019 - accuracy: 0.9420 - val_loss: 3.1516 - val_accuracy: 0.3720
Epoch 120/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.1912 - accuracy: 0.9445 - val_loss: 3.5569 - val_accuracy: 0.3727
Epoch 121/1000
6/6 [==============================] - 1s 137ms/step - loss: 0.2080 - accuracy: 0.9384 - val_loss: 3.5210 - val_accuracy: 0.3727
Epoch 1

Epoch 172/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1831 - accuracy: 0.9462 - val_loss: 3.0912 - val_accuracy: 0.3734
Epoch 173/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1594 - accuracy: 0.9568 - val_loss: 3.4718 - val_accuracy: 0.3727
Epoch 174/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1700 - accuracy: 0.9511 - val_loss: 3.2034 - val_accuracy: 0.3734
Epoch 175/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1673 - accuracy: 0.9537 - val_loss: 3.1943 - val_accuracy: 0.3734
Epoch 176/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1553 - accuracy: 0.9507 - val_loss: 3.3878 - val_accuracy: 0.3727
Epoch 177/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.1688 - accuracy: 0.9519 - val_loss: 3.1565 - val_accuracy: 0.3727
Epoch 178/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.1619 - accuracy: 0.9502 - val_loss: 2.7689 - val_accuracy:

Epoch 229/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1745 - accuracy: 0.9467 - val_loss: 3.4748 - val_accuracy: 0.3734
Epoch 230/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1788 - accuracy: 0.9373 - val_loss: 3.7339 - val_accuracy: 0.3727
Epoch 231/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1631 - accuracy: 0.9509 - val_loss: 3.4999 - val_accuracy: 0.3741
Epoch 232/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1529 - accuracy: 0.9570 - val_loss: 4.0039 - val_accuracy: 0.3727
Epoch 233/1000
6/6 [==============================] - 1s 138ms/step - loss: 0.1519 - accuracy: 0.9518 - val_loss: 3.9058 - val_accuracy: 0.3727
Epoch 234/1000
6/6 [==============================] - 1s 131ms/step - loss: 0.1537 - accuracy: 0.9537 - val_loss: 3.8135 - val_accuracy: 0.3727
Epoch 235/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1396 - accuracy: 0.9608 - val_loss: 4.1361 - val_accuracy:

Epoch 286/1000
6/6 [==============================] - 1s 132ms/step - loss: 0.1397 - accuracy: 0.9573 - val_loss: 3.4139 - val_accuracy: 0.3734
Epoch 287/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1415 - accuracy: 0.9620 - val_loss: 3.5794 - val_accuracy: 0.3734
Epoch 288/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1419 - accuracy: 0.9558 - val_loss: 3.4034 - val_accuracy: 0.3741
Epoch 289/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1358 - accuracy: 0.9591 - val_loss: 3.2309 - val_accuracy: 0.3761
Epoch 290/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.1375 - accuracy: 0.9587 - val_loss: 3.6393 - val_accuracy: 0.3734
Epoch 291/1000
6/6 [==============================] - 1s 132ms/step - loss: 0.1402 - accuracy: 0.9570 - val_loss: 3.4010 - val_accuracy: 0.3734
Epoch 292/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.1288 - accuracy: 0.9603 - val_loss: 3.6221 - val_accuracy:

...

Epoch 742/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0865 - accuracy: 0.9774 - val_loss: 6.3792 - val_accuracy: 0.3747
Epoch 743/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.0803 - accuracy: 0.9774 - val_loss: 6.1155 - val_accuracy: 0.3747
Epoch 744/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0855 - accuracy: 0.9756 - val_loss: 6.5069 - val_accuracy: 0.3734
Epoch 745/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0850 - accuracy: 0.9749 - val_loss: 6.3792 - val_accuracy: 0.3734
Epoch 746/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0850 - accuracy: 0.9756 - val_loss: 6.6232 - val_accuracy: 0.3727
Epoch 747/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0958 - accuracy: 0.9706 - val_loss: 6.1134 - val_accuracy: 0.3741
Epoch 748/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0774 - accuracy: 0.9765 - val_loss: 6.6396 - val_accuracy:

Epoch 799/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0751 - accuracy: 0.9775 - val_loss: 6.5237 - val_accuracy: 0.3741
Epoch 800/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.0735 - accuracy: 0.9788 - val_loss: 6.1232 - val_accuracy: 0.3747
Epoch 801/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0894 - accuracy: 0.9730 - val_loss: 6.5585 - val_accuracy: 0.3741
Epoch 802/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0790 - accuracy: 0.9744 - val_loss: 6.2695 - val_accuracy: 0.3747
Epoch 803/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0748 - accuracy: 0.9772 - val_loss: 6.9088 - val_accuracy: 0.3741
Epoch 804/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0774 - accuracy: 0.9744 - val_loss: 6.7348 - val_accuracy: 0.3747
Epoch 805/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0878 - accuracy: 0.9730 - val_loss: 6.9760 - val_accuracy:

Epoch 856/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0680 - accuracy: 0.9782 - val_loss: 5.9220 - val_accuracy: 0.3747
Epoch 857/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0898 - accuracy: 0.9699 - val_loss: 5.9238 - val_accuracy: 0.3747
Epoch 858/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0741 - accuracy: 0.9796 - val_loss: 6.4422 - val_accuracy: 0.3747
Epoch 859/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.0693 - accuracy: 0.9788 - val_loss: 6.0698 - val_accuracy: 0.3754
Epoch 860/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0775 - accuracy: 0.9781 - val_loss: 6.4061 - val_accuracy: 0.3747
Epoch 861/1000
6/6 [==============================] - 1s 133ms/step - loss: 0.0660 - accuracy: 0.9800 - val_loss: 6.5470 - val_accuracy: 0.3747
Epoch 862/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0752 - accuracy: 0.9763 - val_loss: 6.3930 - val_accuracy:

Epoch 913/1000
6/6 [==============================] - 1s 138ms/step - loss: 0.0558 - accuracy: 0.9840 - val_loss: 5.6522 - val_accuracy: 0.3754
Epoch 914/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0539 - accuracy: 0.9856 - val_loss: 5.5319 - val_accuracy: 0.3754
Epoch 915/1000
6/6 [==============================] - 1s 137ms/step - loss: 0.0606 - accuracy: 0.9808 - val_loss: 5.4597 - val_accuracy: 0.3754
Epoch 916/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0650 - accuracy: 0.9796 - val_loss: 5.5746 - val_accuracy: 0.3754
Epoch 917/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0666 - accuracy: 0.9808 - val_loss: 5.0707 - val_accuracy: 0.3754
Epoch 918/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0740 - accuracy: 0.9763 - val_loss: 5.1596 - val_accuracy: 0.3754
Epoch 919/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0776 - accuracy: 0.9725 - val_loss: 5.6647 - val_accuracy:

Epoch 970/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0599 - accuracy: 0.9829 - val_loss: 6.2460 - val_accuracy: 0.3754
Epoch 971/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0651 - accuracy: 0.9807 - val_loss: 6.1099 - val_accuracy: 0.3754
Epoch 972/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0680 - accuracy: 0.9803 - val_loss: 6.6554 - val_accuracy: 0.3747
Epoch 973/1000
6/6 [==============================] - 1s 135ms/step - loss: 0.0682 - accuracy: 0.9800 - val_loss: 6.3003 - val_accuracy: 0.3747
Epoch 974/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0666 - accuracy: 0.9781 - val_loss: 6.2626 - val_accuracy: 0.3747
Epoch 975/1000
6/6 [==============================] - 1s 136ms/step - loss: 0.0703 - accuracy: 0.9760 - val_loss: 6.1762 - val_accuracy: 0.3747
Epoch 976/1000
6/6 [==============================] - 1s 134ms/step - loss: 0.0560 - accuracy: 0.9849 - val_loss: 5.7411 - val_accuracy:

In [125]:
print(train_features.shape)
print(val_features.shape)

(5744, 1250, 19)
(1457, 1250, 19)


In [126]:
print(train_features.reshape(-1, train_features.shape[-1]).shape)

(7180000, 19)


In [127]:
# Save Model Weights
model.save_weights("Schizophrenia_CNN_Weights")